## Preliminaries

Setting-up

In [1]:
setwd(base::dirname(getwd()))
getwd()

[1] "J:/library/helminthiases/preliminary"

<br>

Libraries

In [ ]:
.activate <- function (x){
    library(x, character.only = TRUE, quietly = TRUE, warn.conflicts = FALSE, verbose = FALSE)
    if (x == 'rmarkdown') {library(tinytex)}
}
log <- lapply(X = c('magrittr', 'tidyr', 'dplyr', 'tibble', 'ggplot2', 'stringr', 'lubridate'), .activate)

<br>

## Data Sources

In [3]:
files <- list.files(path = file.path(getwd(), 'warehouse', 'missing', 'disaggregates'),
                    full.names = TRUE)

<br>
<br>

## Null Correlation

In [4]:
source(file = 'R/missing/Steps.R')

In [5]:
cores <- parallel::detectCores() - 2
doParallel::registerDoParallel(cores = cores)
clusters <- parallel::makeCluster(cores)
X <- parallel::clusterMap(clusters, fun = Steps, files, USE.NAMES = FALSE)
parallel::stopCluster(clusters)
rm(clusters, cores)

In [6]:
correlations <- dplyr::bind_rows(X)

In [7]:
alternative <- tidyr::pivot_longer(data = correlations, 
                                   cols = c(year, longitude, latitude, hk_prevalence, asc_prevalence, tt_prevalence, coordinates), 
                                   names_to = 'reference', values_to = 'cramer')

<br>

### Preserve

Storage

In [8]:
storage <- file.path(getwd(), 'warehouse', 'missing', 'correlation')
if ( dir.exists(storage)) {
  base::unlink(x = storage, recursive = TRUE)
}
dir.create(path = storage, recursive = TRUE)

<br>

Write

In [9]:
utils::write.table(x = correlations,
                   file = file.path(storage, 'wide.csv'),
                   append = FALSE,
                   sep = ',',
                   na = '',
                   row.names = FALSE,
                   col.names = TRUE,
                   fileEncoding = 'UTF-8')

In [10]:
utils::write.table(x = alternative,
                   file = file.path(storage, 'long.csv'),
                   append = FALSE,
                   sep = ',',
                   na = '',
                   row.names = FALSE,
                   col.names = TRUE,
                   fileEncoding = 'UTF-8')